##Run those cells if you have any problem with the GPU

In [1]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
# # # First, clear the current model from memory
# torch.cuda.empty_cache()

# import gc
# gc.collect()

# # # Check memory freed
# !nvidia-smi

# this is our start

In [3]:
# ---------------------------
# CLEAN PROJECT FOLDER SETUP
# ---------------------------
# Why this is here:
# - Every Colab session is clean.
# - We need a stable folder to store data, logs, HITL feedback.

from pathlib import Path

WORKDIR = Path("/content/personalized_chatbot")
WORKDIR.mkdir(exist_ok=True)

DATA_DIR = WORKDIR / "data"
DATA_DIR.mkdir(exist_ok=True)

HITL_FILE = WORKDIR / "feedback.jsonl"      # where Human-in-the-Loop corrections go
FINETUNE_PREP = WORKDIR / "finetune_prep"
FINETUNE_PREP.mkdir(exist_ok=True)

print("WORKDIR:", WORKDIR)
print("DATA_DIR:", DATA_DIR)
print("FINETUNE_PREP:", FINETUNE_PREP)

WORKDIR: /content/personalized_chatbot
DATA_DIR: /content/personalized_chatbot/data
FINETUNE_PREP: /content/personalized_chatbot/finetune_prep


In [4]:
# Cell 1A: Extended directory structure for MLOps pipeline (FIXED)
# Place this RIGHT AFTER your existing Cell 1 (folder setup)

# Create additional directories for pipeline
PROCESSED_IDS_FILE = WORKDIR / "processed_feedback_ids.json"

# Use FINETUNE_PREP from your original notebook instead of creating BATCH_DIR
if 'BATCH_DIR' not in globals():
    BATCH_DIR = FINETUNE_PREP  # Use existing directory

BATCH_DIR.mkdir(exist_ok=True)

# Create empty processed IDs file if doesn't exist
if not PROCESSED_IDS_FILE.exists():
    import json
    with open(PROCESSED_IDS_FILE, 'w') as f:
        json.dump([], f)

print("Extended MLOps directories:")
print(f"  Processed IDs: {PROCESSED_IDS_FILE}")
print(f"  Training batches: {BATCH_DIR}")

Extended MLOps directories:
  Processed IDs: /content/personalized_chatbot/processed_feedback_ids.json
  Training batches: /content/personalized_chatbot/finetune_prep


In [5]:
# LLaMA inference (transformers + bitsandbytes)
# LoRA (peft)
# dataset management (datasets)
# orchestration (langchain)
!pip install -q transformers accelerate bitsandbytes peft datasets langchain sentencepiece

In [6]:
# Cell 2A: Install MLOps dependencies (FIXED for Colab)
# Place this RIGHT AFTER your existing Cell 2 (pip install)

# Prefect has dependency issues on Colab, so we'll use a simpler orchestration approach
# We'll track pipeline runs manually with timestamps and logs

# Install only what we absolutely need
!pip install -q datasets pandas

print("✓ Pipeline dependencies installed")
print("  Note: Using lightweight pipeline tracking instead of Prefect")

✓ Pipeline dependencies installed
  Note: Using lightweight pipeline tracking instead of Prefect


In [7]:
# Cell 2B: Simple Pipeline Orchestration (Prefect Alternative)
# Place after Cell 2A

import json
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Callable
import traceback

class PipelineRun:
    """
    Lightweight pipeline orchestration without external dependencies.

    Tracks:
    - Task execution times
    - Success/failure status
    - Error messages
    - Run metadata
    """

    def __init__(self, name: str, log_dir: Path):
        self.name = name
        self.log_dir = log_dir
        self.log_dir.mkdir(exist_ok=True)

        self.run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.start_time = time.time()
        self.tasks = []
        self.status = "running"

    def run_task(self, task_name: str, task_func: Callable, *args, **kwargs):
        """
        Execute a task with error handling and logging.

        Args:
            task_name: Human-readable task name
            task_func: Function to execute
            *args, **kwargs: Arguments to pass to task_func

        Returns:
            Task result if successful, None if failed
        """
        print(f"\n{'='*60}")
        print(f"Task: {task_name}")
        print(f"{'='*60}")

        task_start = time.time()
        task_record = {
            "name": task_name,
            "start_time": task_start,
            "status": "running"
        }

        try:
            # Execute task
            result = task_func(*args, **kwargs)

            # Record success
            task_record["status"] = "success"
            task_record["duration"] = time.time() - task_start
            task_record["result_summary"] = str(result)[:200] if result else "None"

            print(f"Task completed in {task_record['duration']:.2f}s")

            self.tasks.append(task_record)
            return result

        except Exception as e:
            # Record failure
            task_record["status"] = "failed"
            task_record["duration"] = time.time() - task_start
            task_record["error"] = str(e)
            task_record["traceback"] = traceback.format_exc()

            print(f"Task FAILED: {e}")

            self.tasks.append(task_record)
            self.status = "failed"

            return None

    def finish(self):
        """Complete the pipeline run and save log"""
        self.duration = time.time() - self.start_time

        if self.status != "failed":
            self.status = "completed"

        # Save run log
        log_file = self.log_dir / f"run_{self.run_id}.json"

        run_log = {
            "run_id": self.run_id,
            "pipeline": self.name,
            "status": self.status,
            "start_time": datetime.fromtimestamp(self.start_time).isoformat(),
            "duration_seconds": self.duration,
            "tasks": self.tasks
        }

        with open(log_file, 'w') as f:
            json.dump(run_log, f, indent=2)

        # Print summary
        print(f"\n{'='*60}")
        print(f"PIPELINE SUMMARY")
        print(f"{'='*60}")
        print(f"Run ID: {self.run_id}")
        print(f"Status: {self.status.upper()}")
        print(f"Duration: {self.duration:.2f}s")
        print(f"Tasks: {len(self.tasks)}")

        for task in self.tasks:
            status_label = "[SUCCEEDED]" if task["status"] == "success" else "[FAILED]"
            print(f"  {status_label} {task['name']}: {task.get('duration', 0):.2f}s")

        print(f"\nLog saved: {log_file}")

        return run_log

# Create pipeline logs directory
PIPELINE_LOGS = WORKDIR / "pipeline_logs"
PIPELINE_LOGS.mkdir(exist_ok=True)

print("✓ Pipeline orchestration ready")
print(f"  Logs directory: {PIPELINE_LOGS}")

✓ Pipeline orchestration ready
  Logs directory: /content/personalized_chatbot/pipeline_logs


we will be using the 3B model since we are working on a CPU backend

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch


MODEL = 'pierreramez/Llama-3.2-3B-Instruct-bnb-4bit_finetuned' # 3B model
# MODEL = 'pierreramez/llama3.1-finetuned-v2' # 8B model

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True # Enable CPU offload for 32-bit modules
)

model = AutoModelForCausalLM.from_pretrained(
                                             MODEL,
                                             quantization_config=bnb_config, # Use BitsAndBytesConfig for quantization
                                             device_map={ '': 0 }, # Explicitly map all layers to GPU 0
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True #required for llama
                                            )

model.config.pad_token_id = tokenizer.pad_token_id

# text gen pipeline
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.2, # low temp to make it more deterministic
    top_p=0.9,
    repetition_penalty=1.2
)

print('Model loaded successfully!\n')

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded successfully!



In [9]:
def generate_reply(user_input, history, max_turns=4):
    """
    history = list of dicts: [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}, ...]
    """
    # Truncate history to last max_turns exchanges
    truncated_history = history[-(max_turns * 2):]

    # Add new user message
    messages = truncated_history + [{"role": "user", "content": user_input}]

    # Apply the SAME chat template used during training
    # Ensure inputs are moved to the model's device
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    attention_mask = (inputs != tokenizer.pad_token_id).long()

    with torch.no_grad():
        output = model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            max_new_tokens=128,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode only new tokens
    response = tokenizer.decode(output[0][inputs.shape[1]:], skip_special_tokens=True)
    return response.strip()

In [10]:
import re, json, html

def clean_text(s):
  s = html.unescape(s)
  s= re.sub(r'\s+',' ',s).strip()
  return s

def chat_to_pairs(chat_log, max_user_context=4):
  '''
  chat_log is like:
  [
    {role: 'user', content: '...'},
    {role: 'assistant', content: '...'},
    ...
  ]

  We convert multi-turn chat into supervised training pairs.
  '''

  pairs = []
  for i in range(len(chat_log) - 1):
      if chat_log[i]["role"] == "user" and chat_log[i+1]["role"] == "assistant":
          # Build concise prompt
          ctx_start = max(0, i - max_user_context*2)
          ctx = chat_log[ctx_start:i+1]

          prompt = " ".join(f"{t['role']}: {clean_text(t['content'])}" for t in ctx)
          response = clean_text(chat_log[i+1]["content"])

          pairs.append({"prompt": prompt, "response": response})

  return pairs

example_chat = [
    {"role":"user","content":"Explain normalization."},
    {"role":"assistant","content":"Normalization rescales features to stable ranges."},
    {"role":"user","content":"Show formula."},
    {"role":"assistant","content":"z = (x - µ) / σ"}
]

pairs = chat_to_pairs(example_chat)
pairs

[{'prompt': 'user: Explain normalization.',
  'response': 'Normalization rescales features to stable ranges.'},
 {'prompt': 'user: Explain normalization. assistant: Normalization rescales features to stable ranges. user: Show formula.',
  'response': 'z = (x - µ) / σ'}]

In [11]:
out_path = DATA_DIR / "train_pairs.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for p in pairs:
        f.write(json.dumps(p, ensure_ascii=False) + "\n")

print("Saved training pairs to:", out_path)

Saved training pairs to: /content/personalized_chatbot/data/train_pairs.jsonl


## Human in the loop (HITL) pipeline

In [12]:
import time

def save_interaction(user_input, model_reply, user_correction=None, reason=None):
    """
    Append a single interaction to feedback.jsonl
    The model learns from mistakes later.
    """
    rec = {
        "time": time.time(),
        "user_input": user_input,
        "model_reply": model_reply,
        "user_correction": user_correction,
        "accepted": user_correction is None,
        "reason": reason,
    }

    with open(HITL_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    return rec

print("HITL pipeline ready!")

HITL pipeline ready!


In [13]:
# Cell 7A: FIXED FeedbackManager (REPLACE the old one completely)
# Place after Cell 7 (save_interaction)

import json
import hashlib
from datetime import datetime
from typing import List, Dict, Optional

class FeedbackManager:
    """
    Fixed version that separates reading from processing.

    Key fix: get_new_corrections() now does NOT mark items as processed.
    Only mark_as_processed() does that, which we call AFTER successful training.
    """

    def __init__(self, feedback_file: Path, processed_ids_file: Path):
        self.feedback_file = feedback_file
        self.processed_ids_file = processed_ids_file
        self.processed_ids = self._load_processed_ids()

    def _load_processed_ids(self) -> set:
        """Load IDs of feedback already used for training"""
        if not self.processed_ids_file.exists():
            return set()

        try:
            with open(self.processed_ids_file, 'r') as f:
                data = json.load(f)
                return set(data) if isinstance(data, list) else set()
        except:
            return set()

    def _save_processed_ids(self):
        """Persist processed IDs to disk"""
        with open(self.processed_ids_file, 'w') as f:
            json.dump(list(self.processed_ids), f)

    def _generate_feedback_id(self, interaction: Dict) -> str:
        """Create unique ID from interaction"""
        content = f"{interaction['user_input']}{interaction['time']}"
        return hashlib.md5(content.encode()).hexdigest()

    def get_new_corrections(self) -> List[Dict]:
        """
        Extract NEW corrections (not yet processed).

        CRITICAL: This does NOT mark them as processed!
        Call mark_as_processed() after successful training.
        """
        if not self.feedback_file.exists():
            return []

        new_corrections = []

        with open(self.feedback_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    interaction = json.loads(line.strip())

                    # Generate ID
                    interaction_id = self._generate_feedback_id(interaction)

                    # Skip if already processed
                    if interaction_id in self.processed_ids:
                        continue

                    # Only corrections (accepted=False means user corrected)
                    if interaction.get('accepted') is False and interaction.get('user_correction'):
                        new_corrections.append({
                            'id': interaction_id,
                            'prompt': interaction['user_input'],
                            'response': interaction['user_correction'],
                            'timestamp': interaction['time'],
                            'reason': interaction.get('reason', 'user_correction')
                        })

                except json.JSONDecodeError:
                    continue

        return new_corrections

    def mark_as_processed(self, correction_ids: List[str]):
        """
        Mark corrections as processed after successful training.

        Call this ONLY after training completes successfully.
        """
        self.processed_ids.update(correction_ids)
        self._save_processed_ids()
        print(f"Marked {len(correction_ids)} corrections as processed")

# Initialize feedback manager
feedback_mgr = FeedbackManager(
    feedback_file=HITL_FILE,
    processed_ids_file=PROCESSED_IDS_FILE
)

print("  Fixed FeedbackManager initialized")
print(f"   Tracking: {HITL_FILE}")
print(f"   Already processed: {len(feedback_mgr.processed_ids)} interactions")

  Fixed FeedbackManager initialized
   Tracking: /content/personalized_chatbot/feedback.jsonl
   Already processed: 11 interactions


In [14]:
# Cell 7B: Training Data Preparation
# Place RIGHT AFTER Cell 7A (Feedback Manager)

from datasets import Dataset
import pandas as pd

class TrainingDataPreparator:
    """
    Converts feedback corrections into model training format.

    Critical: Must match the instruction format your model was trained on.
    Your model expects: instruction + input + output structure.
    """

    def __init__(self, output_dir: Path):
        self.output_dir = output_dir
        self.output_dir.mkdir(exist_ok=True)

    def prepare_training_batch(self, corrections: List[Dict]) -> Optional[Path]:
        """
        Transform corrections into training-ready format.

        Format:
        - instruction: The user's question/prompt
        - input: Empty for conversational models
        - output: The corrected response
        """
        if not corrections:
            print("No corrections to prepare")
            return None

        # Convert to instruction-following format
        training_examples = []
        for corr in corrections:
            training_examples.append({
                "messages": [
                    {"role": "user", "content": corr["prompt"]},
                    {"role": "assistant", "content": corr["response"]}
                ],
                "metadata": {
                    "id": corr["id"],
                    "timestamp": corr["timestamp"],
                    "reason": corr.get("reason", "user_correction")
                }
            })

        # Save as JSONL with timestamp
        batch_file = self.output_dir / f"batch_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"

        with open(batch_file, 'w', encoding='utf-8') as f:
            for example in training_examples:
                f.write(json.dumps(example, ensure_ascii=False) + '\n')

        print(f"✓ Prepared {len(training_examples)} training examples")
        print(f"  Saved to: {batch_file}")

        return batch_file

    def create_huggingface_dataset(self, jsonl_file: Path) -> Dataset:
        """
        Load prepared JSONL into HuggingFace Dataset.
        This is what the Trainer expects.
        """
        data = []
        with open(jsonl_file, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))

        df = pd.DataFrame(data)
        dataset = Dataset.from_pandas(df)

        print(f"✓ HuggingFace dataset: {len(dataset)} examples")
        return dataset

# Initialize preparator
data_prep = TrainingDataPreparator(output_dir=BATCH_DIR)

print(" Training data preparator READY")
print(f"  Output directory: {BATCH_DIR}")

 Training data preparator READY
  Output directory: /content/personalized_chatbot/finetune_prep


In [15]:
# Cell 7C: Pipeline Tasks (Updated for lightweight orchestration)
# Place after Cell 7B

def collect_new_feedback() -> List[Dict]:
    """
    Task: Extract new corrections from feedback log.
    """
    print(" Collecting feedback...")
    corrections = feedback_mgr.get_new_corrections()

    if corrections:
        print(f"✓ Found {len(corrections)} new corrections")
        for i, corr in enumerate(corrections[:3], 1):
            print(f"  {i}. {corr['prompt'][:60]}...")
    else:
        print("ℹ️  No new corrections found")

    return corrections

def prepare_data_for_training(corrections: List[Dict]) -> Optional[Path]:
    """
    Task: Format corrections into training dataset.
    """
    if not corrections:
        print("  Skipping - NO CORRECTIONS")
        return None

    print("Preparing training data...")
    batch_file = data_prep.prepare_training_batch(corrections)
    return batch_file

def validate_training_dataset(batch_file: Optional[Path]) -> bool:
    if batch_file is None:
        return False

    print("  Validating dataset...")

    try:
        with open(batch_file, 'r') as f:
            lines = f.readlines()

        if len(lines) == 0:
            print("Dataset is empty")
            return False

        # Check for messages format
        first_ex = json.loads(lines[0])
        if "messages" not in first_ex:
            print(f" Missing 'messages' field. Has: {first_ex.keys()}")
            return False

        # Verify messages structure
        messages = first_ex["messages"]
        if not isinstance(messages, list) or len(messages) < 2:
            print(f" INVALID MESSAGES STRUCTURE")
            return False

        print(f" Dataset VALID ({len(lines)} examples)")
        return True

    except Exception as e:
        print(f" Validation FAILED: {e}")
        return False

print("✓ Pipeline tasks defined")

✓ Pipeline tasks defined


In [16]:
# Cell 7D: Data Pipeline Execution (UPDATED)
# REPLACE the old Cell 7D with this

def run_data_pipeline() -> Dict:
    """
    Execute the data preparation pipeline.

    Now marks corrections as processed AFTER successful preparation.
    """
    pipeline = PipelineRun(
        name="data_preparation",
        log_dir=PIPELINE_LOGS
    )

    print(f"\nSTARTING DATA PREPARATION PIPELINE")
    print(f"Run ID: {pipeline.run_id}\n")

    # Task 1: Collect feedback (does NOT mark as processed yet)
    corrections = pipeline.run_task(
        "collect_feedback",
        collect_new_feedback
    )

    if not corrections:
        pipeline.finish()
        return {
            "status": "no_data",
            "dataset_path": None,
            "num_examples": 0
        }

    # Task 2: Prepare training data
    batch_file = pipeline.run_task(
        "prepare_training_data",
        prepare_data_for_training,
        corrections
    )

    # Task 3: Validate
    is_valid = pipeline.run_task(
        "validate_dataset",
        validate_training_dataset,
        batch_file
    )

    # Finish pipeline
    run_log = pipeline.finish()

    # Mark as processed ONLY if successful
    if is_valid and batch_file:
        correction_ids = [c['id'] for c in corrections]
        feedback_mgr.mark_as_processed(correction_ids)

        return {
            "status": "success",
            "dataset_path": str(batch_file),
            "num_examples": len(corrections),
            "run_id": pipeline.run_id
        }
    else:
        return {
            "status": "failed",
            "dataset_path": None,
            "num_examples": 0,
            "run_id": pipeline.run_id
        }

print(" Data pipeline READY (with proper processing)")
print("\n To run: result = run_data_pipeline()")

 Data pipeline READY (with proper processing)

 To run: result = run_data_pipeline()


##How Fine-Tuning Would Be Done (LoRA Prep) <<DON'T RUN>>

In [17]:
# # WARNING: DO NOT RUN ON COLAB FREE.
# # This is for your Milestone documentation.

# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# from transformers import TrainingArguments, Trainer
# from datasets import load_dataset

# # Load dataset
# train_data_path = str(FINETUNE_PREP / "hitl_for_finetune.jsonl")
# dataset = load_dataset("json", data_files=train_data_path, split="train")

# def tokenize(entry):
#     # Format: "### Prompt" pattern helps the model learn dialog structure
#     inp = "### Prompt:\n" + entry["prompt"] + "\n\n### Response:\n"
#     txt = inp + entry["response"]

#     tok = tokenizer(txt, truncation=True, max_length=512)

#     # Label masking: prompt tokens = -100 (ignored)
#     labels = tok["input_ids"].copy()
#     prompt_len = len(tokenizer(inp)["input_ids"])
#     labels[:prompt_len] = [-100] * prompt_len

#     tok["labels"] = labels
#     return tok

# dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

# # LoRA config
# lora_cfg = LoraConfig(
#     r=8, lora_alpha=32,
#     target_modules=["q_proj","k_proj","v_proj","o_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# lora_model = prepare_model_for_kbit_training(model)
# lora_model = get_peft_model(lora_model, lora_cfg)

# training_args = TrainingArguments(
#     output_dir=str(WORKDIR / "lora_ckpt"),
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8,
#     num_train_epochs=1,
#     learning_rate=2e-4,
#     fp16=True,
#     logging_steps=10
# )

# trainer = Trainer(
#     model=lora_model,
#     args=training_args,
#     train_dataset=dataset
# )

# print("Training pipeline ready (but not running).")

In [18]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA available: True
Device: Tesla T4


## simple chat loop

In [19]:
history = []

while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["exit","quit"]:
        break

    # Before calling generate_reply, ensure model is on the correct device if it wasn't during initial load.
    # The root cause is the model being on CPU despite a GPU being available.
    # The generate_reply function will be modified to send inputs to model.device
    reply = generate_reply(user_input, history)
    print("Assistant:", reply)

    # Log HITL?
    correction = input("Correction? (empty = accepted): ").strip()
    if correction:
        save_interaction(user_input, reply, correction, reason="manual feedback")
    else:
        save_interaction(user_input, reply)

    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": reply})

KeyboardInterrupt: Interrupted by user

### production API configuration

In [20]:
# Cell 23A: Production API Configuration
import os

print("  PRODUCTION API CONFIGURATION")
print("="*70)

# Try environment variable first
PRODUCTION_API_URL = os.getenv('PRODUCTION_API_URL', '')

if PRODUCTION_API_URL and 'YOUR-USERNAME' not in PRODUCTION_API_URL:
    print(f"SUCCESS: Using environment variable")
    print(f"   API URL: {PRODUCTION_API_URL}")
else:
    print("No environment variable found")
    print("\n   Options:")
    print("   1. Enter API URL now (for production)")
    print("   2. Press Enter to skip (local mode only)")

    user_input = input("\n   API URL: ").strip()

    if user_input:
        PRODUCTION_API_URL = user_input
        print(f"    Set to: {PRODUCTION_API_URL}")
    else:
        PRODUCTION_API_URL = ""
        print("    Local mode (no production API)")

# Auto-detect mode
USE_PRODUCTION_API = bool(PRODUCTION_API_URL)

print("\n" + "="*70)
print(f"Mode: {'PRODUCTION' if USE_PRODUCTION_API else 'LOCAL'}")
if USE_PRODUCTION_API:
    print(f"API: {PRODUCTION_API_URL}")
else:
    print("Using local feedback file only")
print("="*70)

  PRODUCTION API CONFIGURATION
No environment variable found

   Options:
   1. Enter API URL now (for production)
   2. Press Enter to skip (local mode only)

   API URL: 
    Local mode (no production API)

Mode: LOCAL
Using local feedback file only


### Download feedback from production

In [21]:
# Cell 23B: Download Feedback from Production (Run before training)
import requests
import json

if not USE_PRODUCTION_API:
    print("LOCAL MODE")
    print("   Using local feedback file (not downloading from production)")
    print(f"   File: {HITL_FILE}")

    if HITL_FILE.exists():
        with open(HITL_FILE, 'r') as f:
            lines = f.readlines()
        print(f"   Interactions: {len(lines)}")

        # Count corrections
        corrections = sum(1 for line in lines
                         if json.loads(line).get('accepted') is False)
        print(f"   Corrections: {corrections}")
    else:
        print("     File doesn't exist yet")
        print("   Run chat widget to create it")

else:
    print(" PRODUCTION MODE")
    print(f"   Downloading feedback from: {PRODUCTION_API_URL}")
    print("")

    try:
        # Check health first
        print("   1. Checking backend health...")
        health_resp = requests.get(f"{PRODUCTION_API_URL}/health", timeout=5)

        if health_resp.status_code != 200:
            print(f"    Backend unhealthy: {health_resp.status_code}")
            print("   Cannot download feedback")
        else:
            print("    Backend is healthy")

            # Check correction count
            print("\n   2. Checking correction count...")
            count_resp = requests.get(
                f"{PRODUCTION_API_URL}/correction-count",
                timeout=10
            )

            if count_resp.status_code == 200:
                data = count_resp.json()
                print(f"   Total interactions: {data['total']}")
                print(f"   Corrections: {data['corrections']}")
                print(f"   Ready to train: {data['ready_to_train']}")

                if data['corrections'] == 0:
                    print("\n     No corrections yet - nothing to download")
                    print("   Users need to provide feedback first")

                elif data['corrections'] < 5:
                    print(f"\n    Not enough corrections yet")
                    print(f"   Minimum: 5, Recommended: 20")
                    print(f"   Missing: {5 - data['corrections']} more")

                else:
                    # Download feedback
                    print("\n   3. Downloading feedback...")
                    download_resp = requests.get(
                        f"{PRODUCTION_API_URL}/download-feedback",
                        timeout=30
                    )

                    if download_resp.status_code == 200:
                        feedback = download_resp.json()

                        # Validate feedback
                        if not feedback['content'].strip():
                            print("     Feedback is empty")
                        else:
                            # Validate JSON format
                            try:
                                lines = feedback['content'].strip().split('\n')
                                valid_count = 0

                                for line in lines[:5]:
                                    json.loads(line)
                                    valid_count += 1

                                print(f"    Validated first {valid_count} entries")

                                # Save to file
                                with open(HITL_FILE, 'w', encoding='utf-8') as f:
                                    f.write(feedback['content'])

                                print(f"    Downloaded {feedback['count']} interactions")
                                print(f"    Saved to: {HITL_FILE}")
                                print("\n    Next: Run Cell 12C to prepare training data")

                            except json.JSONDecodeError as e:
                                print(f"    Invalid JSON format: {e}")
                                print("   Feedback file may be corrupted")
                    else:
                        print(f"    Download failed: {download_resp.status_code}")
            else:
                print(f"    Count check failed: {count_resp.status_code}")

    except requests.exceptions.Timeout:
        print("    Request timed out")
        print("    Backend might be sleeping (free tier)")
        print("   Try again in 30 seconds")

    except requests.exceptions.ConnectionError:
        print("    Cannot connect to backend")
        print("    Check:")
        print("   1. Is PRODUCTION_API_URL correct?")
        print("   2. Is backend Space running?")
        print("   3. Try: curl {url}/health".replace('{url}', PRODUCTION_API_URL))

    except Exception as e:
        print(f"    Unexpected error: {e}")

print("\n" + "="*70)

LOCAL MODE
   Using local feedback file (not downloading from production)
   File: /content/personalized_chatbot/feedback.jsonl
   Interactions: 35
   Corrections: 11



## CELL 12A: Enhanced Data Pipeline with Train/Test Split

In [22]:
# Cell 12A: Enhanced Data Preparation with Train/Test Split
# Place AFTER Cell 11 (chat loop) - BEFORE old Cell 12

from sklearn.model_selection import train_test_split
import json
from pathlib import Path

class EnhancedDataPreparator:
    """
    Splits corrections into train/test sets for proper evaluation.

    Minimum 20 corrections required:
    - 80% (16) for training
    - 20% (4) for evaluation
    """

    def __init__(self, min_corrections: int = 20):
        self.min_corrections = min_corrections

    def prepare_split_datasets(self, corrections: List[Dict]) -> tuple:
        """
        Split corrections and prepare separate train/test files.

        Returns:
            (train_file_path, test_file_path, train_count, test_count)
        """

        print(f"\n Data Split Analysis:")
        print(f"   Total corrections: {len(corrections)}")

        # Validation
        if len(corrections) < self.min_corrections:
            print(f"     WARNING: Need {self.min_corrections} corrections for reliable evaluation")
            print(f"   Current: {len(corrections)} | Missing: {self.min_corrections - len(corrections)}")
            print(f"\n   Options:")
            print(f"   1. Collect more corrections (RECOMMENDED)")
            print(f"   2. Proceed anyway (results may be unreliable)")

            proceed = input("\n   Continue anyway? (y/N): ").strip().lower()
            if proceed != 'y':
                return None, None, 0, 0

        # 80/20 split with stratification (if you had labels)
        # For now, random split with fixed seed for reproducibility
        train_data, test_data = train_test_split(
            corrections,
            test_size=0.2,      # 20% for testing
            random_state=42,    # Reproducible splits
            shuffle=True        # Randomize before splitting
        )

        print(f"\n    Split complete:")
        print(f"      Training set: {len(train_data)} examples ({len(train_data)/len(corrections)*100:.1f}%)")
        print(f"      Test set: {len(test_data)} examples ({len(test_data)/len(corrections)*100:.1f}%)")

        # Prepare training file
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        train_file = BATCH_DIR / f"train_{timestamp}.jsonl"
        test_file = BATCH_DIR / f"test_{timestamp}.jsonl"

        # Save training data (CORRECT format matching your training)
        with open(train_file, 'w', encoding='utf-8') as f:
            for corr in train_data:
                training_example = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': corr['prompt']
                        },
                        {
                            'role': 'assistant',
                            'content': corr['response']
                        }
                    ]
                }
                f.write(json.dumps(training_example, ensure_ascii=False) + '\n')

        # Save test data (CORRECT format matching your training)
        with open(test_file, 'w') as f:
            for corr in test_data:
                test_example = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': corr['prompt']
                        },
                        {
                            'role': 'assistant',
                            'content': corr['response']
                        }
                    ]
                }
                f.write(json.dumps(test_example, ensure_ascii=False) + '\n')

        print(f"\n    Files saved:")
        print(f"      Train: {train_file.name}")
        print(f"      Test: {test_file.name}")

        return train_file, test_file, len(train_data), len(test_data)

# Initialize enhanced preparator
enhanced_prep = EnhancedDataPreparator(min_corrections=20)

print(" Enhanced data preparator ready")
print("   Minimum corrections: 20 (16 train, 4 test)")

 Enhanced data preparator ready
   Minimum corrections: 20 (16 train, 4 test)


## convert HITL logs to fine-tuning dataset in JSONL

In [23]:
def convert_feedback_to_finetune():
    src = HITL_FILE
    out = FINETUNE_PREP / "hitl_for_finetune.jsonl"

    if not Path(src).exists():
        print("No feedback yet.")
        return None

    count = 0
    with open(src, "r", encoding="utf-8") as f, open(out, "w", encoding="utf-8") as out_f:
        for line in f:
            rec = json.loads(line)
            if rec["accepted"] is False and rec["user_correction"]:
                out_f.write(json.dumps({
                    "prompt": rec["user_input"],
                    "response": rec["user_correction"]
                }, ensure_ascii=False) + "\n")
                count += 1

    print(f"Converted {count} corrected samples → {out}")
    return out

convert_feedback_to_finetune()
# we ignore the accepted responses and only keep the corrections

Converted 11 corrected samples → /content/personalized_chatbot/finetune_prep/hitl_for_finetune.jsonl


PosixPath('/content/personalized_chatbot/finetune_prep/hitl_for_finetune.jsonl')

# using custom class to orchestrate the pipeline

In [24]:
# Cell 12B: REPLACE OLD CELL 12 with this Enhanced Pipeline
# This version splits data properly and tracks both train/test files

def run_enhanced_data_pipeline() -> Dict:
    """
    Enhanced pipeline with train/test split.

    Returns dict with:
        - status: 'success', 'no_data', or 'failed'
        - train_dataset_path: Path to training JSONL
        - test_dataset_path: Path to test JSONL
        - num_train_examples: Count
        - num_test_examples: Count
    """

    pipeline = PipelineRun(
        name="enhanced_data_preparation",
        log_dir=PIPELINE_LOGS
    )

    print(f"\n STARTING ENHANCED DATA PIPELINE")
    print(f"Run ID: {pipeline.run_id}\n")

    # Task 1: Collect feedback
    corrections = pipeline.run_task(
        "collect_feedback",
        collect_new_feedback
    )

    if not corrections:
        with mlflow.start_run():
          mlflow.set_tag("status", "no_feedback")
          mlflow.log_metric("num_corrections", 0)

        pipeline.finish()
        return {
            "status": "no_data",
            "train_dataset_path": None,
            "test_dataset_path": None,
            "num_train_examples": 0,
            "num_test_examples": 0
        }

    # Task 2: Split and prepare datasets
    def split_and_save():
        return enhanced_prep.prepare_split_datasets(corrections)

    train_file, test_file, train_count, test_count = pipeline.run_task(
        "split_train_test",
        split_and_save
    )

    if train_file is None:
        pipeline.finish()
        return {
            "status": "insufficient_data",
            "train_dataset_path": None,
            "test_dataset_path": None,
            "num_train_examples": 0,
            "num_test_examples": 0
        }

    # Task 3: Validate both files
    def validate_both():
        train_valid = validate_training_dataset(train_file)
        test_valid = validate_training_dataset(test_file)
        return train_valid and test_valid

    is_valid = pipeline.run_task(
        "validate_datasets",
        validate_both
    )

    # Finish pipeline
    run_log = pipeline.finish()

    # Mark as processed ONLY if successful
    if is_valid and train_file and test_file:
        correction_ids = [c['id'] for c in corrections]
        feedback_mgr.mark_as_processed(correction_ids)

        return {
            "status": "success",
            "train_dataset_path": str(train_file),
            "test_dataset_path": str(test_file),
            "num_train_examples": train_count,
            "num_test_examples": test_count,
            "run_id": pipeline.run_id
        }
    else:
        return {
            "status": "failed",
            "train_dataset_path": None,
            "test_dataset_path": None,
            "num_train_examples": 0,
            "num_test_examples": 0,
            "run_id": pipeline.run_id
        }

print(" Enhanced pipeline ready")
print("\n To run: result = run_enhanced_data_pipeline()")

 Enhanced pipeline ready

 To run: result = run_enhanced_data_pipeline()


diagnostic cell

In [42]:
# DIAGNOSTIC CELL: Check Your Feedback Status
# Run this to understand what's in your feedback file

if 'HITL_FILE' not in dir():
    print(" HITL_FILE not defined")
    print("   Please run Cells 4-5 first to set up folders")
else:
  import json
  from pathlib import Path

  print("FEEDBACK DIAGNOSTICS")
  print("="*70)

  # Check if feedback file exists
  if HITL_FILE.exists():
      print(f" Feedback file found: {HITL_FILE}")
      print(f"   File size: {HITL_FILE.stat().st_size} bytes")

      # Read and analyze feedback
      with open(HITL_FILE, 'r', encoding='utf-8') as f:
          lines = f.readlines()

      total = len(lines)
      corrections = 0
      accepted = 0
      errors = 0

      print(f"\n ANALYZING {total} INTERACTIONS...")
      print("-"*70)

      # Show first 3 interactions
      print("\n First 3 interactions:")
      for i, line in enumerate(lines[:3], 1):
          try:
              record = json.loads(line.strip())
              print(f"\n   {i}. User: {record.get('user_input', 'N/A')[:60]}...")
              print(f"      Bot:  {record.get('model_reply', 'N/A')[:60]}...")
              print(f"      Accepted: {record.get('accepted', 'N/A')}")
              if not record.get('accepted', True):
                  print(f"      Correction: {record.get('user_correction', 'N/A')[:60]}...")
                  corrections += 1
              else:
                  accepted += 1
          except Exception as e:
              print(f"   {i}.  Error parsing: {e}")
              errors += 1

      # Count all interactions
      corrections = 0
      accepted = 0
      errors = 0

      for line in lines:
          try:
              record = json.loads(line.strip())
              if record.get('accepted') is False:
                  corrections += 1
              else:
                  accepted += 1
          except:
              errors += 1

      # Summary
      print("\n" + "="*70)
      print(" SUMMARY:")
      print(f"   Total interactions: {total}")
      print(f"   Corrections (accepted=False): {corrections}")
      print(f"   Accepted (accepted=True or None): {accepted}")
      print(f"   Parse errors: {errors}")

      # Analysis
      print("\n" + "="*70)
      print(" ANALYSIS:")

      if corrections == 0:
          print("     PROBLEM: You have NO corrections!")
          print("")
          print("   This means:")
          print("   • You chatted with the bot")
          print("   • But you pressed Enter without typing corrections")
          print("   • The chat loop treats empty input as 'accepted'")
          print("")
          print("    SOLUTION:")
          print("   1. Go back to the chat loop (Cell 11 or 23)")
          print("   2. Chat with the bot")
          print("   3. When bot gives a WRONG answer, TYPE a correction")
          print("   4. Press Enter to submit the correction")
          print("   5. Come back and run Cell 12C again")

      elif corrections < 5:
          print(f"     Very few corrections: {corrections}")
          print(f"   You need at least 5 to train, 20 for good results")
          print(f"   Missing: {max(5, 20) - corrections} more")
          print("")
          print("   Keep chatting and providing corrections!")

      elif corrections < 20:
          print(f"    You have {corrections} corrections")
          print(f"   This is enough to train, but more is better")
          print(f"   Recommended: Collect {20 - corrections} more for reliable evaluation")
          print("")
          print("    You can proceed to Cell 12C now!")

      else:
          print(f"    Excellent! You have {corrections} corrections")
          print(f"   This is great for training!")
          print("")
          print("    Proceed to Cell 12C to prepare training data")

  else:
      print(f" Feedback file NOT found: {HITL_FILE}")
      print("")
      print("   This means you haven't chatted with the bot yet.")
      print("")
      print("    SOLUTION:")
      print("   1. Run the chat loop cell (Cell 11 or 23)")
      print("   2. Chat with the bot")
      print("   3. Provide corrections when needed")
      print("   4. Come back and run this diagnostic again")

  print("\n" + "="*70)
  print(" NEXT STEPS:")

  if not HITL_FILE.exists() or corrections == 0:
      print("   1. Chat with bot and provide corrections")
      print("   2. Run this diagnostic again")
      print("   3. When you have 5+ corrections, run Cell 12C")
      print("   4. Then run Cell 13G to train")
  elif corrections < 5:
      print(f"   1. Collect {5 - corrections} more corrections (minimum)")
      print("   2. Run this diagnostic again")
      print("   3. When ready, run Cell 12C")
      print("   4. Then run Cell 13G to train")
  else:
      print("   1. Run Cell 12C to prepare training data")
      print("   2. Check that result['status'] == 'success'")
      print("   3. Run Cell 13G to train and evaluate")
      print("   4. Check MLflow for results")

  print("="*70)


FEEDBACK DIAGNOSTICS
 Feedback file found: /content/personalized_chatbot/feedback.jsonl
   File size: 22796 bytes

 ANALYZING 35 INTERACTIONS...
----------------------------------------------------------------------

 First 3 interactions:

   1. User: h...
      Bot:  It seems like you just sent a single character. Is there som...
      Accepted: True

   2. User: h...
      Bot:  It seems like you just sent a single character. Would you li...
      Accepted: True

   3. User: h...
      Bot:  It seems like you just sent a single character. Is there som...
      Accepted: True

 SUMMARY:
   Total interactions: 35
   Corrections (accepted=False): 11
   Accepted (accepted=True or None): 24
   Parse errors: 0

 ANALYSIS:
    You have 11 corrections
   This is enough to train, but more is better
   Recommended: Collect 9 more for reliable evaluation

    You can proceed to Cell 12C now!

 NEXT STEPS:
   1. Run Cell 12C to prepare training data
   2. Check that result['status'] == 'success

In [45]:
import json

# Clear the processed IDs file
processed_file = WORKDIR / "processed_feedback_ids.json"

# Check current state
if processed_file.exists():
    with open(processed_file, 'r') as f:
        processed = json.load(f)
    print(f"Currently marked as processed: {len(processed)} corrections")

    # Reset it
    with open(processed_file, 'w') as f:
        json.dump([], f)
    print("✓ Reset processed IDs - corrections are available again!")
else:
    print("No processed IDs file found")

# Reload the feedback manager
feedback_mgr = FeedbackManager(
    feedback_file=HITL_FILE,
    processed_ids_file=PROCESSED_IDS_FILE
)

# Check again
available = feedback_mgr.get_new_corrections()
print(f"\n✓ Available corrections now: {len(available)}")

No processed IDs file found

✓ Available corrections now: 11


In [46]:
# Cell 12C: Execute Enhanced Pipeline (FIXED)
# Place AFTER Cell 12B

print("🔍 Checking for corrections...")

# Check available corrections
available_corrections = feedback_mgr.get_new_corrections()
print(f" Available corrections: {len(available_corrections)}")

if len(available_corrections) == 0:
    print("\n  No corrections found!")
    print("   Instructions:")
    print("   1. Go back to Cell 11 (chat loop)")
    print("   2. Chat with the bot")
    print("   3. When it gives wrong answers, provide corrections")
    print("   4. Come back here and run this cell again")

    # FIX: Set result even when there are no corrections
    result = {
        "status": "no_data",
        "train_dataset_path": None,
        "test_dataset_path": None,
        "num_train_examples": 0,
        "num_test_examples": 0
    }

elif len(available_corrections) < 20:
    print(f"\n  Only {len(available_corrections)} corrections")
    print(f"   Recommended: Collect at least 20 for reliable evaluation")
    print(f"   Missing: {20 - len(available_corrections)} more corrections")
    print("\n   You can proceed, but results may be unreliable")

    # Run anyway
    result = run_enhanced_data_pipeline()

    if result["status"] == "success":
        print(f"\n Pipeline completed despite low data count")
        print(f"   Training examples: {result['num_train_examples']}")
        print(f"   Test examples: {result['num_test_examples']}")

else:
    print(f"\n Sufficient corrections: {len(available_corrections)}")
    print("   Running pipeline...\n")

    # Run pipeline
    result = run_enhanced_data_pipeline()

    if result["status"] == "success":
        print(f"\n SUCCESS!")
        print(f"   Training data: {result['train_dataset_path']}")
        print(f"   Test data: {result['test_dataset_path']}")
        print(f"   Train examples: {result['num_train_examples']}")
        print(f"   Test examples: {result['num_test_examples']}")
        print(f"\n Next: Scroll down to training cells")
    else:
        print(f"\n  Pipeline status: {result['status']}")

🔍 Checking for corrections...
 Available corrections: 11

  Only 11 corrections
   Recommended: Collect at least 20 for reliable evaluation
   Missing: 9 more corrections

   You can proceed, but results may be unreliable

 STARTING ENHANCED DATA PIPELINE
Run ID: 20251130_164919


Task: collect_feedback
✓ Found 11 new corrections
  1. Give me C++ bubble sort code, but do not show any code...
  2. Explain supervised vs unsupervised learning without talking ...
  3. Explain how diffusion transformers outperform CNN-RNN system...
Task completed in 0.00s

Task: split_train_test

 Data Split Analysis:
   Total corrections: 11
   Current: 11 | Missing: 9

   Options:
   1. Collect more corrections (RECOMMENDED)
   2. Proceed anyway (results may be unreliable)

   Continue anyway? (y/N): y

    Split complete:
      Training set: 8 examples (72.7%)
      Test set: 3 examples (27.3%)

    Files saved:
      Train: train_20251130_164926.jsonl
      Test: test_20251130_164926.jsonl
Task complete

## LoRA training optimized for T4 GPU

In [47]:
# Cell 13A: Training Configuration for T4 GPU

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from transformers import BitsAndBytesConfig
import torch
from typing import Optional
import os

epochs = 2
batch_size = 1
gradient_accumulation_steps = 8
learning_rate = 2e-4
lora_alpha = 32
lora_dropout = 0.05

class LoRATrainingConfig:
    """
    Training configuration optimized for Colab T4 GPU (15GB VRAM).

    Key optimizations:
    - 4-bit quantization reduces memory by ~75%
    - Small LoRA rank (r=8) keeps adapter tiny
    - Gradient accumulation simulates larger batch sizes
    - Aggressive gradient checkpointing saves memory
    """

    def __init__(self, dataset_path: str, output_dir: Path):
        self.dataset_path = dataset_path
        self.output_dir = output_dir

        # LoRA Configuration
        # r=8 means we're adding 8-rank decomposition matrices
        # Lower r = less parameters = less memory = faster training
        # But too low = model can't learn enough
        self.lora_config = LoraConfig(
            r=8,                          # Rank (8 is sweet spot for T4)
            lora_alpha=lora_alpha,                # Scaling factor (typically 2-4x rank)
            target_modules=[              # Which layers to adapt
                "q_proj",                 # Query projection
                "k_proj",                 # Key projection
                "v_proj",                 # Value projection
                "o_proj"                  # Output projection
            ],
            lora_dropout=lora_dropout,            # Regularization
            bias="none",                  # Don't adapt bias terms
            task_type=TaskType.CAUSAL_LM  # Causal language modeling
        )

        # 4-bit Quantization Config
        # This is THE trick that makes 8B models fit on T4
        self.bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,                    # Enable 4-bit
            bnb_4bit_quant_type="nf4",           # "Normal Float 4" - best quality
            bnb_4bit_compute_dtype=torch.float16, # Compute in FP16
            bnb_4bit_use_double_quant=True       # Double quantization saves more memory
        )

        # Training Arguments
        # These are VERY conservative to avoid OOM
        self.training_args = TrainingArguments(
            output_dir=str(output_dir),

            # Batch size: Start tiny!
            per_device_train_batch_size=batch_size,        # Only 1 example at a time
            gradient_accumulation_steps=gradient_accumulation_steps,        # But accumulate 8 steps = effective batch of 8

            # Epochs: 1-2 is enough for small datasets
            num_train_epochs=epochs,

            # Learning rate: Higher than normal because LoRA needs strong signal
            learning_rate=learning_rate,                   # 0.0002

            # Memory optimizations
            fp16=True,                            # Use half precision
            gradient_checkpointing=True,          # Trade compute for memory
            optim="paged_adamw_8bit",            # 8-bit optimizer (saves 4GB!)

            # Logging and checkpointing
            logging_steps=5,                      # Log every 5 steps
            save_strategy="epoch",                # Save after each epoch
            save_total_limit=2,                   # Keep only 2 checkpoints

            # Misc
            warmup_steps=10,                      # Gradual learning rate warmup
            report_to=[],                         # Don't report to wandb/tensorboard
            remove_unused_columns=False,          # Keep all columns for debugging
        )

    def get_memory_usage(self):
        """Check current GPU memory usage"""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**3  # Convert to GB
            reserved = torch.cuda.memory_reserved() / 1024**3
            return f"Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB"
        return "CUDA not available"

# Initialize config
training_config = None  # We'll set this in the next cell

print("✓ Training configuration class defined")
print("\n Optimizations for T4 GPU:")
print("  • 4-bit quantization (saves ~10GB)")
print("  • LoRA rank 8 (keeps adapter small)")
print("  • Batch size 1 + gradient accumulation 8")
print("  • 8-bit optimizer (saves ~4GB)")
print("  • Gradient checkpointing enabled")

✓ Training configuration class defined

 Optimizations for T4 GPU:
  • 4-bit quantization (saves ~10GB)
  • LoRA rank 8 (keeps adapter small)
  • Batch size 1 + gradient accumulation 8
  • 8-bit optimizer (saves ~4GB)
  • Gradient checkpointing enabled


### Dataset tokenization

In [48]:
# Cell 13B: Dataset Preparation and Tokenization (CHAT FORMAT)

def prepare_training_dataset(dataset_path: str, tokenizer):
    from datasets import load_dataset

    print(f" Loading dataset from: {dataset_path}")
    dataset = load_dataset("json", data_files=dataset_path, split="train")
    print(f"   Loaded {len(dataset)} examples")

    #  Verify format
    print("\n Sample (messages format):")
    print(dataset[0])

    # Apply chat template
    def apply_chat_template(example):
        formatted = tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
        return {"text": formatted}

    print("\n Applying chat template...")
    dataset = dataset.map(
        apply_chat_template,
        remove_columns=dataset.column_names
    )

    # Tokenization
    def tokenize(example):
        encoded = tokenizer(
            example["text"],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_tensors=None
        )
        encoded["labels"] = encoded["input_ids"].copy()
        return encoded

    print(" Tokenizing...")
    tokenized_dataset = dataset.map(
        tokenize,
        batched=True,
        remove_columns=["text"],
        desc="Tokenizing"
    )

    print(f" Tokenized {len(tokenized_dataset)} examples")
    return tokenized_dataset

print("✓ Chat-format dataset preparation function defined")

✓ Chat-format dataset preparation function defined


### Main training function

In [49]:
# Cell 13C: Main Training Function (FIXED)

def train_lora_adapter(
    dataset_path: str,
    output_name: str = "llama-lora-adapter",
    push_to_hub: bool = True,
    hub_repo_name: Optional[str] = None
):
    """
    Train a LoRA adapter on corrected examples.

    Args:
        dataset_path: Path to prepared JSONL dataset
        output_name: Local directory name for saving
        push_to_hub: Whether to push to Hugging Face Hub
        hub_repo_name: HF repo name (e.g., "yourusername/model-name")

    Returns:
        Path to trained adapter
    """
    global lora_training_loss
    global training_config
    print("\n" + "="*70)
    print(" STARTING LORA TRAINING")
    print("="*70)

    # Setup output directory
    adapter_output_dir = WORKDIR / output_name
    adapter_output_dir.mkdir(exist_ok=True)

    # Initialize config

    training_config = LoRATrainingConfig(
        dataset_path=dataset_path,
        output_dir=adapter_output_dir
    )

    print(f"\n Initial GPU memory: {training_config.get_memory_usage()}")

    # Step 1: Load tokenizer (already loaded, but ensure padding token)
    print("\n Setting up tokenizer...")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print(f"   Pad token: {tokenizer.pad_token}")

    # Step 2: Load and tokenize dataset
    print("\n Preparing dataset...")
    train_dataset = prepare_training_dataset(dataset_path, tokenizer)

    if len(train_dataset) < 5:
        print("\n  WARNING: Less than 5 examples!")
        print("   Training might not improve the model meaningfully.")
        print("   Recommended: Collect at least 20 corrections.")
        proceed = input("\n   Continue anyway? (y/N): ").strip().lower()
        if proceed != 'y':
            print(" Training cancelled")
            return None

    # Step 3: Prepare model for training
    print("\n Preparing model for LoRA training...")
    print("   This will take 2-3 minutes...")

    print("\n Clearing GPU memory...")
    import gc

    # Clear cache first
    torch.cuda.empty_cache()
    gc.collect()

    print(f"   Memory freed: {torch.cuda.memory_allocated() / 1024**3:.2f}GB")

    # Prepare model for training (model is still on GPU)
    model_for_training = prepare_model_for_kbit_training(model)

    # Ensure model is on GPU
    if torch.cuda.is_available():
        model_for_training = model_for_training.to('cuda')

    print(f"   GPU memory after prep: {training_config.get_memory_usage()}")
    print(f"   Model device: {next(model_for_training.parameters()).device}")

    # Step 4: Add LoRA adapters
    print("\n Adding LoRA adapters...")
    lora_model = get_peft_model(model_for_training, training_config.lora_config)

    # Print trainable parameters
    trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in lora_model.parameters())
    trainable_percent = 100 * trainable_params / total_params

    print(f"   Trainable params: {trainable_params:,} ({trainable_percent:.2f}%)")
    print(f"   Total params: {total_params:,}")
    print(f"   Memory: {training_config.get_memory_usage()}")

    # Step 5: Create trainer
    print("\n  Creating trainer...")

    # Data collator handles batching and padding
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # We're doing causal LM, not masked LM
    )

    trainer = Trainer(
        model=lora_model,
        args=training_config.training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
    )

    # Step 6: Train
    print("\n Starting training...")
    print("   This will take 10-20 minutes depending on dataset size.")
    print("   Watch the loss - it should decrease.\n")

    try:
        #  FIX: Train first then save loss
        train_result = trainer.train()

        print("\n TRAINING COMPLETED!")
        print(f"   Final loss: {train_result.training_loss:.4f}")

        #  Save loss AFTER training (when train_result exists)
        lora_training_loss = train_result.training_loss

    except RuntimeError as e:
        if "out of memory" in str(e):
            print("\n OUT OF MEMORY ERROR!")
            print("\n Try these fixes:")
            print("   1. Reduce per_device_train_batch_size to 1 (if not already)")
            print("   2. Increase gradient_accumulation_steps to 16")
            print("   3. Reduce max_length to 256 in tokenization")
            print("   4. Use a smaller LoRA rank (r=4)")
            print("\n   Restart runtime and try again.")

            # Set default loss value on error
            lora_training_loss = float('inf')
            return None
        else:
            raise e

    # Step 7: Save adapter
    print(f"\n Saving adapter to {adapter_output_dir}...")
    lora_model.save_pretrained(adapter_output_dir)
    tokenizer.save_pretrained(adapter_output_dir)

    print("✓ Adapter saved locally")

    # Step 8: Push to Hugging Face Hub (optional)
    if push_to_hub:
        if hub_repo_name is None:
            print("\n  No hub_repo_name provided. Skipping push to HF Hub.")
            print("   To push later, run:")
            print(f"   huggingface-cli upload {hub_repo_name} {adapter_output_dir}")
        else:
            print(f"\n  Pushing to Hugging Face Hub: {hub_repo_name}")
            try:
                lora_model.push_to_hub(hub_repo_name)
                tokenizer.push_to_hub(hub_repo_name)
                print(f" Pushed to https://huggingface.co/{hub_repo_name}")
            except Exception as e:
                print(f"  Push failed: {e}")
                print("   You can push manually later using huggingface-cli")

    print("\n" + "="*70)
    print(" TRAINING COMPLETE!")
    print("="*70)
    print(f"Adapter location: {adapter_output_dir}")
    print(f"Memory used: {training_config.get_memory_usage()}")

    return adapter_output_dir

print(" Training function defined")
print("\n Ready to train! See Cell 13G for execution.")

 Training function defined

 Ready to train! See Cell 13G for execution.


####T4 GPU memory optimizations

In [50]:
# Cell 12D: T4 Memory Optimization
def optimize_for_t4():
    """Aggressive memory optimization for T4 GPU (15GB)."""
    import gc

    print(" Optimizing for T4 GPU...")

    # Clear cache
    torch.cuda.empty_cache()
    gc.collect()

    # Set memory allocation strategy
    torch.cuda.set_per_process_memory_fraction(0.95)  # Use 95% max

    # Enable TF32 for faster training (if supported)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

    print(f"Optimized - Available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")

optimize_for_t4()

 Optimizing for T4 GPU...
Optimized - Available: 14.7GB


## Multi-Metric Evaluator

In [51]:
# Cell 13E: Multi-Metric Model Evaluator

!pip install -q sentence-transformers rouge-score nltk

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt', quiet=True)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

class ModelEvaluator:
    """
    Multi-metric evaluation for fine-tuned models.

    Metrics:
    1. Semantic Similarity (embedding distance)
    2. ROUGE-L (word overlap with order)
    3. BLEU (n-gram precision)
    4. Exact Match (strict correctness)
    """

    def __init__(self):
        print(" Loading evaluation models...")

        # Lightweight embedding model (22MB)
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

        # ROUGE scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

        # BLEU smoothing (handles edge cases)
        self.smoothing = SmoothingFunction()

        print(" Evaluation models loaded")

    def generate_response(self, model, tokenizer, prompt: str) -> str:
      """Generate using SAME format as training."""
      messages = [{'role': 'user', 'content': prompt}]

      # Apply chat template (same as training)
      formatted = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True  # Add assistant prompt
      )

      inputs = tokenizer(formatted, return_tensors="pt").to(model.device)

      with torch.no_grad():
          outputs = model.generate(
              **inputs,
              max_new_tokens=128,
              temperature=0.7,
              do_sample=True,
              pad_token_id=tokenizer.pad_token_id
          )

      # Decode only new tokens
      response = tokenizer.decode(
          outputs[0][inputs['input_ids'].shape[1]:],
          skip_special_tokens=True
      )

      return response.strip()

    def calculate_metrics(self, generated: str, ground_truth: str) -> Dict[str, float]:
        """Calculate all metrics for one example."""

        # 1. Semantic Similarity (embedding-based)
        embeddings = self.embedding_model.encode([generated, ground_truth])
        semantic_sim = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

        # 2. ROUGE-L (word overlap with longest common subsequence)
        rouge_scores = self.rouge_scorer.score(ground_truth, generated)
        rouge_l = rouge_scores['rougeL'].fmeasure

        # 3. BLEU (n-gram precision)
        reference = [ground_truth.split()]
        candidate = generated.split()
        bleu_score = sentence_bleu(
            reference,
            candidate,
            smoothing_function=self.smoothing.method1
        )

        # 4. Exact Match (binary - exact correctness)
        exact_match = int(generated.strip().lower() == ground_truth.strip().lower())

        return {
            'semantic_similarity': float(semantic_sim),
            'rouge_l': float(rouge_l),
            'bleu': float(bleu_score),
            'exact_match': exact_match
        }

    def evaluate_model(
        self,
        model,
        tokenizer,
        test_file: Path,
        model_name: str = "Model"
    ) -> Dict:
        """
        Evaluate model on test set.

        Returns:
            Dictionary with averaged metrics and per-example details
        """

        print(f"\n{'='*70}")
        print(f" EVALUATING: {model_name}")
        print(f"{'='*70}")
        print(f"Test file: {test_file.name}\n")

        # Load test data
        test_examples = []
        with open(test_file, 'r') as f:
            for line in f:
                test_examples.append(json.loads(line))

        print(f"Test examples: {len(test_examples)}\n")

        # Collect metrics
        all_metrics = {
            'semantic_similarity': [],
            'rouge_l': [],
            'bleu': [],
            'exact_match': []
        }

        detailed_results = []

        for i, example in enumerate(test_examples, 1):
            # Extract from messages format
            messages = example['messages']
            prompt = next(m['content'] for m in messages if m['role'] == 'user')
            ground_truth = next(m['content'] for m in messages if m['role'] == 'assistant')

            # Generate
            print(f"[{i}/{len(test_examples)}] Generating...", end=" ")
            generated = self.generate_response(model, tokenizer, prompt)

            # Calculate metrics
            metrics = self.calculate_metrics(generated, ground_truth)

            # Store
            for key, value in metrics.items():
                all_metrics[key].append(value)

            detailed_results.append({
                'prompt': prompt,
                'generated': generated,
                'ground_truth': ground_truth,
                'metrics': metrics
            })

            # Print summary
            print(f"Semantic: {metrics['semantic_similarity']:.3f} | ROUGE-L: {metrics['rouge_l']:.3f}")

        # Calculate averages
        avg_metrics = {key: np.mean(values) for key, values in all_metrics.items()}

        print(f"\n{'='*70}")
        print(f" SUMMARY: {model_name}")
        print(f"{'='*70}")
        print(f"Semantic Similarity: {avg_metrics['semantic_similarity']:.4f}")
        print(f"ROUGE-L:            {avg_metrics['rouge_l']:.4f}")
        print(f"BLEU:               {avg_metrics['bleu']:.4f}")
        print(f"Exact Match:        {avg_metrics['exact_match']:.4f} ({int(avg_metrics['exact_match']*len(test_examples))}/{len(test_examples)})")
        print(f"{'='*70}\n")

        return {
            'average_metrics': avg_metrics,
            'detailed_results': detailed_results,
            'num_test_examples': len(test_examples)
        }

# Initialize evaluator
evaluator = ModelEvaluator()

print("\n Multi-metric evaluator ready")
print("   Metrics: Semantic Similarity, ROUGE-L, BLEU, Exact Match")

 Loading evaluation models...
 Evaluation models loaded

 Multi-metric evaluator ready
   Metrics: Semantic Similarity, ROUGE-L, BLEU, Exact Match


DAGSHUB preparation

In [52]:
# DagsHub Authentication
import os
from google.colab import userdata

# Option 1: Use Colab Secrets
try:
    DAGSHUB_USERNAME = userdata.get('DAGSHUB_USERNAME')
    DAGSHUB_TOKEN = userdata.get('DAGSHUB_TOKEN')
    print(" Using Colab secrets for DagsHub authentication")
except:
    # Option 2: Manual input
    print("  Colab secrets not found, using manual input")
    DAGSHUB_USERNAME = input("Enter DagsHub username: ")
    DAGSHUB_TOKEN = input("Enter DagsHub token: ")

# Set environment variables for MLflow
os.environ['MLFLOW_TRACKING_USERNAME'] = DAGSHUB_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = DAGSHUB_TOKEN

print(f" Authenticated as: {DAGSHUB_USERNAME}")
print(" Ready to log to DagsHub MLflow")

 Using Colab secrets for DagsHub authentication
 Authenticated as: PierreRamez
 Ready to log to DagsHub MLflow


## MLFlow integration with Evaluation

In [55]:
# Cell 13F: MLflow Integration with Proper Evaluation
!pip install mlflow
import mlflow
import mlflow.pytorch
import numpy as np

def convert_to_serializable(obj):
    """Convert numpy types and bools to JSON-serializable types."""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, bool):
        return int(obj)
    else:
        return obj

# Configure MLflow
mlflow.set_experiment('Llama-3.2-3B-Personalized-Chatbot')
mlflow.set_tracking_uri('https://dagshub.com/PierreRamez/PersonalChatbot.mlflow')

def train_and_evaluate_with_mlflow(
    train_dataset_path: str,
    test_dataset_path: str,
    output_name: str = "llama-lora-adapter",
    push_to_hub: bool = True,
    hub_repo_name: str = None
):
    """
    Complete training + evaluation pipeline with MLflow tracking.

    This is the CORRECT way to integrate MLflow:
    1. Start run BEFORE training
    2. Log params, train, evaluate
    3. Log all metrics and artifacts
    4. Conditionally register model based on quality
    """

    # Validate inputs
    train_path = Path(train_dataset_path)
    test_path = Path(test_dataset_path)

    if not train_path.exists() or not test_path.exists():
        print(" Train or test file missing!")
        return None

    print("\n" + "="*70)
    print(" TRAINING WITH MLFLOW TRACKING")
    print("="*70)

    # Start MLflow run (to wrap everything)
    with mlflow.start_run() as run:
        run_id = run.info.run_id
        print(f"\n MLflow Run ID: {run_id}")
        print(f" Track at: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow")

        # ===== PHASE 1: LOG HYPERPARAMETERS =====
        print(f"\n{'='*70}")
        print(" PHASE 1: Logging Hyperparameters")
        print(f"{'='*70}")

        mlflow.log_param("base_model", MODEL)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("lora_rank", 8)
        mlflow.log_param("lora_alpha", lora_alpha)
        mlflow.log_param("lora_dropout", lora_dropout)
        mlflow.log_param("gradient_accumulation_steps", gradient_accumulation_steps)
        mlflow.log_param("max_seq_length", 512)

        print(" Hyperparameters logged")

        # ===== PHASE 2: TRAIN MODEL =====
        print(f"\n{'='*70}")
        print(" PHASE 2: Training")
        print(f"{'='*70}")

        adapter_path = train_lora_adapter(
            dataset_path=train_dataset_path,
            output_name=output_name,
            push_to_hub=False  # We'll push conditionally later
        )

        if adapter_path is None:
            print("\n Training failed!")
            mlflow.set_tag("status", "training_failed")
            return None

        # Log training loss (from global variable set during training)
        if 'lora_training_loss' in globals():
            mlflow.log_metric("train_loss", lora_training_loss)
            print(f" Training loss logged: {lora_training_loss:.4f}")

        # ===== PHASE 3: EVALUATE BASELINE =====
        print(f"\n{'='*70}")
        print(" PHASE 3: Evaluating Baseline Model")
        print(f"{'='*70}")

        # Load original model (without adapter)
        baseline_results = evaluator.evaluate_model(
            model,  # Original model from Cell 3
            tokenizer,
            test_path,
            model_name="Baseline (Original)"
        )

        # Log baseline metrics
        for metric_name, value in baseline_results['average_metrics'].items():
            mlflow.log_metric(f"baseline_{metric_name}", value)

        # ===== PHASE 4: EVALUATE FINE-TUNED =====
        print(f"\n{'='*70}")
        print(" PHASE 4: Evaluating Fine-Tuned Model")
        print(f"{'='*70}")

        # Load fine-tuned model
        from peft import PeftModel
        finetuned_model = PeftModel.from_pretrained(
            model,
            adapter_path,
            torch_dtype=torch.float16
        )

        finetuned_results = evaluator.evaluate_model(
            finetuned_model,
            tokenizer,
            test_path,
            model_name="Fine-Tuned"
        )

        # Log fine-tuned metrics
        for metric_name, value in finetuned_results['average_metrics'].items():
            mlflow.log_metric(f"finetuned_{metric_name}", value)

        # ===== PHASE 5: CALCULATE IMPROVEMENT =====
        print(f"\n{'='*70}")
        print(" PHASE 5: Improvement Analysis")
        print(f"{'='*70}")

        improvements = {}
        for metric in baseline_results['average_metrics'].keys():
            baseline_val = baseline_results['average_metrics'][metric]
            finetuned_val = finetuned_results['average_metrics'][metric]
            improvement = finetuned_val - baseline_val
            improvements[metric] = improvement

            mlflow.log_metric(f"improvement_{metric}", improvement)

            # Print comparison
            trend_label = "[IMPROVED]" if improvement > 0 else "[DECLINED]"
            print(f"{trend_label} {metric:20s}: {baseline_val:.4f} -> {finetuned_val:.4f} (Diff: {improvement:+.4f})")

        # ===== PHASE 6: QUALITY GATE =====
        print(f"\n{'='*70}")
        print("PHASE 6: Quality Gate Check")
        print(f"{'='*70}")

        # Define thresholds
        QUALITY_GATES = {
            'rouge_l': 0.05,  # Must improve by at least 5%
            'semantic_similarity': 0.05,  # Must improve by at least 5%
        }

        # Check if model passes all gates
        gates_passed = {}
        for metric, threshold in QUALITY_GATES.items():
            passed = improvements[metric] >= threshold
            gates_passed[metric] = passed

            status = "PASS" if passed else "FAIL"
            print(f"{status} {metric}: {improvements[metric]:+.4f} (threshold: {threshold:+.4f})")

            mlflow.log_metric(f"gate_{metric}_passed", int(passed))

        all_gates_passed = all(gates_passed.values())
        mlflow.log_metric("all_gates_passed", int(all_gates_passed))

        # ===== PHASE 7: CONDITIONAL DEPLOYMENT =====
        print(f"\n{'='*70}")
        print("PHASE 7: Deployment Decision")
        print(f"{'='*70}")

        if all_gates_passed:
            print("All quality gates PASSED!")
            print("   Model is ready for deployment")

            mlflow.set_tag("status", "approved")
            mlflow.set_tag("deployment_decision", "deploy")

            # Log the adapter as MLflow artifact
            # Log the adapter as MLflow artifact
            mlflow.log_artifacts(adapter_path, artifact_path="adapter")

            # Register model (may not be supported by DagsHub)
            try:
                model_uri = f"runs:/{run_id}/adapter"
                model_version = mlflow.register_model(model_uri, "chatbot-model-prod")
                print(f"\nModel registered: {model_version.name} v{model_version.version}")
            except Exception as e:
                print(f"\n⚠️ Model registration skipped (not supported by DagsHub)")
                print(f"   Your adapter is saved as an artifact in the run")

            # Push to HuggingFace if requested
            if push_to_hub and hub_repo_name:
                print(f"\nPushing to HuggingFace: {hub_repo_name}")
                try:
                    finetuned_model.push_to_hub(hub_repo_name)
                    tokenizer.push_to_hub(hub_repo_name)
                    mlflow.set_tag("huggingface_repo", hub_repo_name)
                    print(f" SUCCESS: Pushed to https://huggingface.co/{hub_repo_name}")
                except Exception as e:
                    print(f"FAILED: HuggingFace push failed: {e}")

        else:
            print("Quality gates FAILED")
            print("   Model will NOT be deployed")
            print("\n   Failed gates:")
            for metric, passed in gates_passed.items():
                if not passed:
                    print(f"      • {metric}: {improvements[metric]:+.4f} < {QUALITY_GATES[metric]:+.4f}")

            mlflow.set_tag("status", "rejected")
            mlflow.set_tag("deployment_decision", "do_not_deploy")

        # ===== PHASE 8: SAVE EVALUATION REPORT =====
        print(f"\n{'='*70}")
        print("PHASE 8: Saving Evaluation Report")
        print(f"{'='*70}")

        # Create detailed report
        report = {
            'run_id': run_id,
            'timestamp': datetime.now().isoformat(),
            'hyperparameters': {
                'epochs': epochs,
                'learning_rate': learning_rate,
                'lora_rank': 8,
                'lora_alpha': lora_alpha,
            },
            'baseline_metrics': baseline_results['average_metrics'],
            'finetuned_metrics': finetuned_results['average_metrics'],
            'improvements': improvements,
            'quality_gates': {
                'passed': int(all_gates_passed),
                'individual_results': {k: int(v) for k, v in gates_passed.items()}
            },
            'deployment_decision': 'deploy' if all_gates_passed else 'do_not_deploy',
            'detailed_test_results': {
                'baseline': baseline_results['detailed_results'],
                'finetuned': finetuned_results['detailed_results']
            }
        }

        # Save report
        report_path = WORKDIR / f"evaluation_report_{run_id}.json"
        with open(report_path, 'w') as f:
            json.dump(convert_to_serializable(report), f, indent=2)

        mlflow.log_artifact(report_path)
        print(f"Report saved: {report_path.name}")

        # ===== FINAL SUMMARY =====
        print(f"\n{'='*70}")
        print("PIPELINE COMPLETE")
        print(f"{'='*70}")
        print(f"Run ID: {run_id}")
        print(f"Status: {'APPROVED' if all_gates_passed else 'REJECTED'}")
        print(f"Decision: {'Deploy to Production' if all_gates_passed else 'Do Not Deploy'}")
        print(f"\nView results: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow#/experiments/{run_id}")

        return {
            'run_id': run_id,
            'status': 'approved' if all_gates_passed else 'rejected',
            'adapter_path': adapter_path if all_gates_passed else None,
            'improvements': improvements,
            'gates_passed': gates_passed
        }

print("MLflow training pipeline ready")
print("\n This function:")
print("   • Trains the model")
print("   • Evaluates baseline vs fine-tuned")
print("   • Applies quality gates")
print("   • Conditionally registers model")
print("   • Generates detailed report")

MLflow training pipeline ready

 This function:
   • Trains the model
   • Evaluates baseline vs fine-tuned
   • Applies quality gates
   • Conditionally registers model
   • Generates detailed report


### execute the complete pipeline

In [56]:
# Cell 13G: Execute Complete Training + Evaluation Pipeline

# Check if we have data from Cell 12C
if 'result' not in globals() or result['status'] != 'success':
    print("No training data available!")
    print("\nPlease run Cell 12C first to prepare data")
else:
    print("Pipeline Configuration:")
    print(f"   Training examples: {result['num_train_examples']}")
    print(f"   Test examples: {result['num_test_examples']}")
    print(f"   Train file: {Path(result['train_dataset_path']).name}")
    print(f"   Test file: {Path(result['test_dataset_path']).name}")

    # Ask for HuggingFace push
    print("\n  Push to Hugging Face Hub after approval?")
    push = input("   (y/N): ").strip().lower() == 'y'

    hub_repo = None
    if push:
        hub_repo = input("   Enter repo name (e.g., 'username/model-name'): ").strip()
        if not hub_repo:
            print("     No repo name - will skip HuggingFace push")
            push = False

    print(f"\n{'='*70}")
    print(" STARTING COMPLETE PIPELINE")
    print(f"{'='*70}")
    print("This will:")
    print("   1. Train LoRA adapter")
    print("   2. Evaluate baseline model")
    print("   3. Evaluate fine-tuned model")
    print("   4. Check quality gates")
    print("   5. Conditionally deploy")
    print(f"\n   Expected time: {result['num_train_examples'] * 2 * 0.5:.0f}-{result['num_train_examples'] * 2:.0f} minutes")
    print(f"{'='*70}")

    confirm = input("\n  Start pipeline? (y/N): ").strip().lower()

    if confirm == 'y':
        # Clear GPU memory
        torch.cuda.empty_cache()

        # RUN THE COMPLETE PIPELINE
        final_result = train_and_evaluate_with_mlflow(
            train_dataset_path=result['train_dataset_path'],
            test_dataset_path=result['test_dataset_path'],
            output_name="llama-lora-adapter",
            push_to_hub=push,
            hub_repo_name=hub_repo if push else None
        )

        if final_result:
            print(f"\n{'='*70}")
            print(" PIPELINE EXECUTION COMPLETE")
            print(f"{'='*70}")
            print(f"Status: {final_result['status'].upper()}")
            print(f"Run ID: {final_result['run_id']}")

            if final_result['status'] == 'approved':
                print(f"\n Model APPROVED for production!")
                print(f"   Adapter saved: {final_result['adapter_path']}")
                print(f"\n Next steps:")
                print(f"   1. Test the deployed model")
                print(f"   2. Update your production inference")
                print(f"   3. Monitor performance in production")
            else:
                print(f"\n  Model REJECTED - did not meet quality standards")
                print(f"\n   Recommendations:")
                print(f"   1. Collect more diverse corrections")
                print(f"   2. Adjust hyperparameters")
                print(f"   3. Review failed quality gates above")
    else:
        print("\nPipeline cancelled")

Pipeline Configuration:
   Training examples: 8
   Test examples: 3
   Train file: train_20251130_164926.jsonl
   Test file: test_20251130_164926.jsonl

  Push to Hugging Face Hub after approval?
   (y/N): y
   Enter repo name (e.g., 'username/model-name'): pierreramez/Llama-3.2-3B-Instruct-bnb-4bit_finetuned

 STARTING COMPLETE PIPELINE
This will:
   1. Train LoRA adapter
   2. Evaluate baseline model
   3. Evaluate fine-tuned model
   4. Check quality gates
   5. Conditionally deploy

   Expected time: 8-16 minutes

  Start pipeline? (y/N): y

 TRAINING WITH MLFLOW TRACKING

 MLflow Run ID: d98905f476874593a2fbd27b42c1c9aa
 Track at: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow

 PHASE 1: Logging Hyperparameters
 Hyperparameters logged

 PHASE 2: Training

 STARTING LORA TRAINING

 Initial GPU memory: Allocated: 2.97GB, Reserved: 4.56GB

 Setting up tokenizer...
   Pad token: <|finetune_right_pad_id|>

 Preparing dataset...
 Loading dataset from: /content/personalized_chatb

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

 Tokenizing...


Tokenizing:   0%|          | 0/8 [00:00<?, ? examples/s]

 Tokenized 8 examples

 Preparing model for LoRA training...
   This will take 2-3 minutes...

 Clearing GPU memory...
   Memory freed: 2.94GB
   GPU memory after prep: Allocated: 2.94GB, Reserved: 4.56GB
   Model device: cuda:0

 Adding LoRA adapters...


...

  ...adapter_model.safetensors:   3%|3         |  568kB / 18.4MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mptb56uipl/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

 SUCCESS: Pushed to https://huggingface.co/pierreramez/Llama-3.2-3B-Instruct-bnb-4bit_finetuned

PHASE 8: Saving Evaluation Report
Report saved: evaluation_report_d98905f476874593a2fbd27b42c1c9aa.json

PIPELINE COMPLETE
Run ID: d98905f476874593a2fbd27b42c1c9aa
Status: APPROVED
Decision: Deploy to Production

View results: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow#/experiments/d98905f476874593a2fbd27b42c1c9aa
🏃 View run silent-fox-759 at: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow/#/experiments/0/runs/d98905f476874593a2fbd27b42c1c9aa
🧪 View experiment at: https://dagshub.com/PierreRamez/PersonalChatbot.mlflow/#/experiments/0

 PIPELINE EXECUTION COMPLETE
Status: APPROVED
Run ID: d98905f476874593a2fbd27b42c1c9aa

 Model APPROVED for production!
   Adapter saved: /content/personalized_chatbot/llama-lora-adapter

 Next steps:
   1. Test the deployed model
   2. Update your production inference
   3. Monitor performance in production


### Upload New adapter

In [57]:
# Cell 23C: Deploy New Adapter (Run after Cell 13G)
import requests

# Check if there's an approved model
if 'final_result' not in globals():
    print("No training result found")
    print("Run Cell 13G first to train a model")
elif not final_result:
    print("Training result is None (training may have failed)")
elif final_result.get('status') != 'approved':
    print(f"Model status: {final_result.get('status')}")
    if final_result.get('status') == 'rejected':
        print("   Model did not pass quality gates")
        print("   Collect more diverse corrections and retrain")
else:
    # Model approved!
    print("Model APPROVED for deployment!")
    print(f"Run ID: {final_result.get('run_id')}")
    print(f"Adapter: {final_result.get('adapter_path')}")

    # Check production API
    if 'USE_PRODUCTION_API' not in globals():
        print("\n USE_PRODUCTION_API not defined")
        print("Run Cell 23A first")
    elif not USE_PRODUCTION_API:
        print("\nProduction API not enabled (USE_PRODUCTION_API = False)")
        print("\n   To deploy:")
        print("   1. Deploy backend to HuggingFace Spaces")
        print("   2. Set USE_PRODUCTION_API = True in Cell 23A")
        print("   3. Run this cell again")
        print("\n   Or manually update your backend:")
        print("   • Go to your backend Space")
        print("   • Edit app.py")
        print("   • Update adapter_path in startup_event()")
    else:
        # Production API enabled
        adapter_name = input("\nEnter HF Hub adapter name (e.g., 'username/adapter-v2'): ").strip()

        if not adapter_name:
            print("No adapter name provided - skipping deployment")
        else:
            print(f"\nDeploying adapter: {adapter_name}")
            print("   This will hot-reload your production backend...\n")

            try:
                # Hot reload
                reload_resp = requests.post(
                    f"{PRODUCTION_API_URL}/reload-adapter",
                    json={"adapter_path": adapter_name},
                    timeout=60
                )

                if reload_resp.status_code == 200:
                    result_data = reload_resp.json()
                    print("Production backend updated successfully!")
                    print(f"Current adapter: {result_data.get('adapter')}")
                    print("New model is now serving requests")

                    # Ask to clear feedback
                    print("\n Clear production feedback to start fresh cycle?")
                    clear = input("   This deletes old corrections (y/N): ").strip()

                    if clear.lower() == 'y':
                        try:
                            clear_resp = requests.post(
                                f"{PRODUCTION_API_URL}/clear-feedback",
                                timeout=10
                            )
                            if clear_resp.status_code == 200:
                                print("Production feedback cleared")
                                print("Ready to collect new corrections")
                            else:
                                print(f"Clear failed: {clear_resp.status_code}")
                        except Exception as e:
                            print(f"Error clearing feedback: {e}")
                    else:
                        print("Keeping existing feedback")

                else:
                    print(f"Reload failed: {reload_resp.status_code}")
                    print(f"Response: {reload_resp.text}")
                    print("\nManual deployment:")
                    print("   1. Go to your backend Space on HuggingFace")
                    print("   2. Edit app.py")
                    print(f"   3. Set adapter_path='{adapter_name}'")
                    print("   4. Commit changes")

            except requests.exceptions.Timeout:
                print("Request timed out")
                print("Backend might be reloading - check Space logs")
            except requests.exceptions.ConnectionError:
                print("Cannot connect to backend")
                print("Check if PRODUCTION_API_URL is correct")
            except Exception as e:
                print(f"Error: {e}")

Model APPROVED for deployment!
Run ID: d98905f476874593a2fbd27b42c1c9aa
Adapter: /content/personalized_chatbot/llama-lora-adapter

Production API not enabled (USE_PRODUCTION_API = False)

   To deploy:
   1. Deploy backend to HuggingFace Spaces
   2. Set USE_PRODUCTION_API = True in Cell 23A
   3. Run this cell again

   Or manually update your backend:
   • Go to your backend Space
   • Edit app.py
   • Update adapter_path in startup_event()


##Test the Trained Adapter

In [58]:
# Cell 14: Test Trained Adapter (FIXED for Messages Format)

def test_adapter_inference(adapter_path: Path, test_messages: list):
    """
    Load the trained adapter and test it on a conversation.

    This shows you if the training actually improved the model.

    Args:
        adapter_path: Path to the LoRA adapter
        test_messages: List of messages in format [{"role": "user", "content": "..."}]
    """
    from peft import PeftModel

    print("\nLoading adapter for testing...")

    # Load adapter on top of base model
    test_model = PeftModel.from_pretrained(
        model,  # Base model from Cell 3
        adapter_path,
        torch_dtype=torch.float16
    )

    print("SUCCESS: Adapter loaded")

    # Format using chat template
    print(f"\nTest conversation:")
    for msg in test_messages:
        role_label = "USER" if msg["role"] == "user" else "ASSISTANT"
        print(f"   {role_label} {msg['role'].title()}: {msg['content']}")

    print("\n🤖 Generating response...\n")

    # Use chat template
    try:
        formatted_prompt = tokenizer.apply_chat_template(
            test_messages,
            tokenize=False,
            add_generation_prompt=True
        )
    except:
        # Fallback if chat template not available
        formatted_prompt = ""
        for msg in test_messages:
            if msg["role"] == "user":
                formatted_prompt += f"User: {msg['content']}\n"
            else:
                formatted_prompt += f"Assistant: {msg['content']}\n"
        formatted_prompt += "Assistant:"

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = test_model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )

    # Decode full response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the new response (after the prompt)
    response_text = full_response[len(formatted_prompt):].strip()

    print("🤖 Response:", response_text)
    print("\n" + "="*70)

    return response_text


# Check if adapter exists
adapter_path = WORKDIR / "llama-lora-adapter"

if not adapter_path.exists():
    print("No adapter found. Train first using Cell 13G.")
else:
    print("Adapter found!")
    print(f"Location: {adapter_path}")

    # Check if we have test data from Cell 12C
    if 'result' in globals() and result.get('test_dataset_path'):
        test_file = Path(result['test_dataset_path'])

        if test_file.exists():
            print(f"\nLoading test example from your corrections...")

            # Read first test example
            with open(test_file, 'r') as f:
                first_example = json.loads(f.readline())

            # Extract messages
            if 'messages' in first_example:
                messages = first_example['messages']

                # Split into input (all but last) and expected output (last)
                input_messages = [msg for msg in messages if msg['role'] == 'user']
                expected_outputs = [msg for msg in messages if msg['role'] == 'assistant']

                # Show what we're testing
                print("\nTest case from your corrections:")
                print("="*70)

                for i, msg in enumerate(messages[:-1], 1):
                    role_label = "USER" if msg["role"] == "user" else "ASSISTANT"
                    print(f"{i}. {role_label} {msg['role'].title()}: {msg['content'][:80]}...")

                if expected_outputs:
                    print(f"\nExpected output (what you corrected to):")
                    print(f"   {expected_outputs[-1]['content'][:150]}...")

                print("\n" + "="*70)

                # Test with just the user messages (let model generate assistant response)
                test_messages = messages[:-1]  # All but the last message

                # Run inference
                result_text = test_adapter_inference(adapter_path, test_messages)

                print("\nCompare the response to your expected output.")
                print("   Does it match better than before training?")

                # Simple similarity check
                if expected_outputs:
                    expected = expected_outputs[-1]['content'].lower()
                    actual = result_text.lower()

                    # Very basic similarity (word overlap)
                    expected_words = set(expected.split())
                    actual_words = set(actual.split())
                    overlap = expected_words & actual_words

                    if len(expected_words) > 0:
                        similarity = len(overlap) / len(expected_words)
                        print(f"\nWord overlap: {similarity*100:.1f}%")

                        if similarity > 0.5:
                            print("Good match!")
                        elif similarity > 0.3:
                            print("Partial match")
                        else:
                            print("Low match - may need more training")
            else:
                print("Unexpected data format in test file")
        else:
            print(f"Test file not found: {test_file}")
            print("\nRun Cell 12C first to prepare test data")
    else:
        print("\nNo test data available")
        print("   Run Cell 12C first to prepare train/test datasets")

    print("\n" + "="*70)
    print("You can also test manually:")
    print("test_adapter_inference(")
    print("adapter_path,")
    print("[{'role': 'user', 'content': 'Your question here'}]")
    print(")")

Adapter found!
Location: /content/personalized_chatbot/llama-lora-adapter

Loading test example from your corrections...

Test case from your corrections:
1. USER User: Describe the loss landscape of a self-dual neural manifold under Symplectic SGD....

Expected output (what you corrected to):
   This entire construct is fictional and undefined....


Loading adapter for testing...


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


SUCCESS: Adapter loaded

Test conversation:
   USER User: Describe the loss landscape of a self-dual neural manifold under Symplectic SGD.

🤖 Generating response...

🤖 Response: 


Compare the response to your expected output.
   Does it match better than before training?

Word overlap: 0.0%
Low match - may need more training

You can also test manually:
test_adapter_inference(
adapter_path,
[{'role': 'user', 'content': 'Your question here'}]
)


### Load Adapter for production use

In [59]:
# Cell 15: Load Adapter for Production Inference

from peft import PeftModel

def load_model_with_adapter(base_model, adapter_path: Path):
    """
    Load the adapter for production use.

    This replaces your original model with the fine-tuned version.
    """
    print(f"Loading adapter from {adapter_path}...")

    adapted_model = PeftModel.from_pretrained(
        base_model,
        adapter_path,
        torch_dtype=torch.float16
    )

    # Merge adapter into base model for faster inference (optional)
    print("Merging adapter with base model...")
    merged_model = adapted_model.merge_and_unload()

    print("Model updated with your corrections!")
    return merged_model

# Load adapter if exists
adapter_path = WORKDIR / "llama-lora-adapter"

if adapter_path.exists():
    print("Adapter found! Loading into model...")

    # Update the global 'model' variable
    model = load_model_with_adapter(model, adapter_path)

    print("\nPRODUCTION MODEL UPDATED!")
    print("   Your chat loop (Cell 11) now uses the fine-tuned model.")
    print("   Go back to Cell 11 and test it!")
else:
    print("No adapter found. Train first (Cell 13D).")

Adapter found! Loading into model...
Loading adapter from /content/personalized_chatbot/llama-lora-adapter...
Merging adapter with base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Model updated with your corrections!

PRODUCTION MODEL UPDATED!
   Your chat loop (Cell 11) now uses the fine-tuned model.
   Go back to Cell 11 and test it!


In [60]:
import requests
import os
PRODUCTION_API_URL  = "https://pierreramez-chatbot-api.hf.space"
print(f"Testing connection to: {PRODUCTION_API_URL}...")

try:
    # 1. Check Health (Basic connectivity)
    health_resp = requests.get(f"{PRODUCTION_API_URL}/health", timeout=15)

    if health_resp.status_code == 200:
        data = health_resp.json()
        print("\nSUCCESS! Connected to backend.")
        print(f"   Status: {data.get('status')}")
        print(f"   Device: {data.get('device')}")
        print(f"   Current Model: {data.get('current_adapter') or 'Base Model'}")

        # 2. Check Feedback Endpoint (Permissions check)
        count_resp = requests.get(f"{PRODUCTION_API_URL}/correction-count", timeout=10)
        if count_resp.status_code == 200:
            count_data = count_resp.json()
            print(f"\nProduction Stats:")
            print(f"Total Interactions: {count_data.get('total')}")
            print(f"Corrections available: {count_data.get('corrections')}")
        else:
            print(f"\nConnected, but '/correction-count' failed: {count_resp.status_code}")

    else:
        print(f"\nBackend is running but returned error: {health_resp.status_code}")
        print(health_resp.text)

except requests.exceptions.ConnectionError:
    print("\nConnection Failed.")
    print("1. Check if the URL in the previous cell is correct.")
    print("2. Go to your Space and ensure it shows 'Running' (Green badge).")
    print("3. If it says 'Building' or 'Runtime Error', check the Space Logs.")

except Exception as e:
    print(f"\nUnexpected Error: {e}")

Testing connection to: https://pierreramez-chatbot-api.hf.space...

SUCCESS! Connected to backend.
   Status: healthy
   Device: cpu
   Current Model: pierreramez/Llama-3.2-3B-Instruct-bnb-4bit_finetuned

Production Stats:
Total Interactions: 34
Corrections available: 1
